In [1]:
# NEED FOR NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#NEED FOR TENSORFLOW
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random
from platform import python_version
print(python_version())

3.8.3


In [2]:
# import chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']

# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        
        # add to our words list
        words.extend(w)
        
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

#documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

#words = all words, vocabulary
print (len(words), "unique stemmed words", words)

54 documents
11 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'end_of_service', 'g6_ps4_pc', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
96 unique stemmed words ["'s", ',', 'a', 'advers', 'al', 'and', 'anyon', 'ar', 'at', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'connect', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'g6', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hey', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'nee', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pc', 'pd1170', 'pharm', 'press', 'provid', 'ps4', 'react', 'rel', 'result', 'sam', 'sb0410', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'the', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'vf0050', 'vf0400', 'want', 'what', 'which', 'with', 'you']


In [4]:
# creating our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialise our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # stem each word - CREATE BASE WORD, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag ( for each pattern )
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

    print(pattern_words)

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

['hi', 'ther']
['how', 'ar', 'you']
['is', 'anyon', 'ther', '?']
['hello']
['good', 'day']
['hey', 'ther']
['hey']
['bye']
['see', 'you', 'lat']
['goodby']
['nic', 'chat', 'to', 'you', ',', 'bye']
['til', 'next', 'tim']
['thank']
['thank', 'you']
['that', "'s", 'help']
['awesom', ',', 'thank']
['thank', 'for', 'help', 'me']
['how', 'you', 'could', 'help', 'me', '?']
['what', 'you', 'can', 'do', '?']
['what', 'help', 'you', 'provid', '?']
['how', 'you', 'can', 'be', 'help', '?']
['what', 'support', 'is', 'off']
['i', 'nee', 'help']
['help', 'me']
['sb0410']
['pd1170']
['vf0050']
['vf0400']
['can', 'you', 'connect', 'the', 'ps4', 'and', 'pc', 'at', 'the', 'sam', 'tim', 'to', 'the', 'g6', '?']
['how', 'to', 'check', 'advers', 'drug', 'react', '?']
['op', 'advers', 'drug', 'mod']
['giv', 'me', 'a', 'list', 'of', 'drug', 'caus', 'advers', 'behavy']
['list', 'al', 'drug', 'suit', 'for', 'paty', 'with', 'advers', 'react']
['which', 'drug', 'dont', 'hav', 'advers', 'react', '?']
['op', 'blood'

In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons
# and 3rd output layer contains number of neurons
# Equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [6]:
# Compile model. Stochastic gradient descent with Nesterov accelerated
# gradient gives good results for this model
sgd = SGD(lr=0.01, decay = 1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [7]:
# Fit the model 
# NOTE : History Object is returned from calls to the fit() function used to train the model
history_obj = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
11/11 [==============================] - 0s 770us/step - loss: 2.4497 - accuracy: 0.0000e+00
Epoch 2/200
11/11 [==============================] - 0s 823us/step - loss: 2.3492 - accuracy: 0.1296
Epoch 3/200
11/11 [==============================] - 0s 652us/step - loss: 2.2209 - accuracy: 0.2407
Epoch 4/200
11/11 [==============================] - 0s 711us/step - loss: 2.1523 - accuracy: 0.2778
Epoch 5/200
11/11 [==============================] - 0s 679us/step - loss: 1.9853 - accuracy: 0.2778
Epoch 6/200
11/11 [==============================] - 0s 926us/step - loss: 1.9684 - accuracy: 0.3148
Epoch 7/200
11/11 [==============================] - 0s 835us/step - loss: 1.7698 - accuracy: 0.4444
Epoch 8/200
11/11 [==============================] - 0s 901us/step - loss: 1.6355 - accuracy: 0.5741
Epoch 9/200
11/11 [==============================] - 0s 680us/step - loss: 1.5217 - accuracy: 0.5370
Epoch 10/200
11/11 [==============================] - 0s 657us/step - loss: 1.4419 - ac

11/11 [==============================] - 0s 721us/step - loss: 0.0881 - accuracy: 0.9815
Epoch 82/200
11/11 [==============================] - 0s 660us/step - loss: 0.0545 - accuracy: 1.0000
Epoch 83/200
11/11 [==============================] - 0s 702us/step - loss: 0.0702 - accuracy: 1.0000
Epoch 84/200
11/11 [==============================] - 0s 664us/step - loss: 0.0625 - accuracy: 0.9815
Epoch 85/200
11/11 [==============================] - 0s 698us/step - loss: 0.0400 - accuracy: 0.9815
Epoch 86/200
11/11 [==============================] - 0s 729us/step - loss: 0.0674 - accuracy: 0.9630
Epoch 87/200
11/11 [==============================] - 0s 661us/step - loss: 0.0401 - accuracy: 1.0000
Epoch 88/200
11/11 [==============================] - 0s 591us/step - loss: 0.0594 - accuracy: 1.0000
Epoch 89/200
11/11 [==============================] - 0s 603us/step - loss: 0.1607 - accuracy: 0.9444
Epoch 90/200
11/11 [==============================] - 0s 677us/step - loss: 0.0919 - accuracy: 

11/11 [==============================] - 0s 574us/step - loss: 0.0280 - accuracy: 1.0000
Epoch 161/200
11/11 [==============================] - 0s 609us/step - loss: 0.0723 - accuracy: 0.9630
Epoch 162/200
11/11 [==============================] - 0s 734us/step - loss: 0.0585 - accuracy: 0.9815
Epoch 163/200
11/11 [==============================] - 0s 622us/step - loss: 0.0183 - accuracy: 1.0000
Epoch 164/200
11/11 [==============================] - 0s 696us/step - loss: 0.0270 - accuracy: 1.0000
Epoch 165/200
11/11 [==============================] - 0s 639us/step - loss: 0.0413 - accuracy: 1.0000
Epoch 166/200
11/11 [==============================] - 0s 549us/step - loss: 0.0168 - accuracy: 1.0000
Epoch 167/200
11/11 [==============================] - 0s 583us/step - loss: 0.0349 - accuracy: 1.0000
Epoch 168/200
11/11 [==============================] - 0s 594us/step - loss: 0.0152 - accuracy: 1.0000
Epoch 169/200
11/11 [==============================] - 0s 699us/step - loss: 0.0232 - a

In [8]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    
    return sentence_words

# RETURN bag of words array : 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" %w)
    
    return (np.array(bag))

In [9]:
p = bow("Load blood pressure for patient", words)
print (p)
print (classes)

found in bag: load
found in bag: blood
found in bag: press
found in bag: for
found in bag: paty
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'end_of_service', 'g6_ps4_pc', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [10]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])
print(model.predict(inputvar))

[[5.89100047e-10 3.33109739e-07 9.99999166e-01 2.00357425e-10
  1.01705515e-08 2.32031561e-10 7.15208934e-11 3.89259839e-07
  3.19547971e-10 4.00167828e-08 2.86169310e-08]]


In [11]:
# Evaluate the model 
#scores = model.evaluate(np.array(train_x), np.array(train_y), verbose=0)
#print("%s: %.2f%% :" % (model.metrics_names[1], scores[1]*100))

model.save("tensorflow-assistant-model.h5")
print("Saved model to disk")

#pickle.dump(history_obj.history, open("tensorflow-assistant-model.pkl", "wb"))

print(history_obj.history.keys())

#pickle.dump(model,open("tensorflow-assistant-model.pkl", "wb"))

Saved model to disk
dict_keys(['loss', 'accuracy'])


In [12]:
# save all of our data structures
pickle.dump( {'words': words, 'classes':classes, 'train_x':train_x, 'train_y':train_y},
           open ("tensorflow-assistant-data.pkl", "wb"))